In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# import nltk
# nltk.download('stopwords')

In [3]:
# print(stopwords.words('english'))

In [4]:
news_dataset = pd.read_csv('train.csv')

In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
news_dataset = news_dataset.fillna("")

In [9]:
#using author and title column
news_dataset['content'] = news_dataset['author'] + news_dataset['title']

In [10]:
news_dataset['content']

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [11]:
stemmer = PorterStemmer()

In [12]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [13]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [14]:
news_dataset['content']

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object

In [15]:
content = news_dataset['content'].values
labels = news_dataset['label'].values

In [16]:
content

array(['darrel lucushous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynnflynn hillari clinton big woman campu breitbart',
       'consortiumnew comwhi truth might get fire', ...,
       'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time',
       'alex ansarynato russia hold parallel exercis balkan',
       'david swansonwhat keep f aliv'], dtype=object)

In [17]:
labels

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [18]:
vectorizer = TfidfVectorizer()
vectorizer.fit(content)
content = vectorizer.transform(content)

In [19]:
print(content)

  (0, 578)	0.2694167078545384
  (0, 4211)	0.36253203231506576
  (0, 5006)	0.24725958235728157
  (0, 5969)	0.35488202138141456
  (0, 6273)	0.2839932825877812
  (0, 8022)	0.2313366174248873
  (0, 12782)	0.24619727512767192
  (0, 14555)	0.2917725968420029
  (0, 15019)	0.4300622675963931
  (0, 22724)	0.25523360180691607
  (0, 26340)	0.2808837940159642
  (1, 2622)	0.3562953366945267
  (1, 3281)	0.18652439327549428
  (1, 3859)	0.45980466668763476
  (1, 4767)	0.23338756776626793
  (1, 5916)	0.31810058109638056
  (1, 8772)	0.5258635625386451
  (1, 11313)	0.24166773097712638
  (1, 27923)	0.36911845953845024
  (2, 5121)	0.5511414848555652
  (2, 5240)	0.40440534260277944
  (2, 8567)	0.3411947414020896
  (2, 9454)	0.30743020569262086
  (2, 16361)	0.43295215406038445
  (2, 26235)	0.3665032495181434
  :	:
  (20797, 1249)	0.3072223353708335
  (20797, 2257)	0.3357782642976524
  (20797, 6088)	0.21253094503918346
  (20797, 11692)	0.2992170910232368
  (20797, 14104)	0.22761807337911874
  (20797, 16217)	0

In [20]:
content_train, content_eval, content_train_labels, content_eval_labels = train_test_split(content, labels, test_size=0.2, stratify=labels, random_state=2)

In [21]:
model = LogisticRegression()

In [22]:
model.fit(content_train, content_train_labels)

LogisticRegression()

In [23]:
#Train data accuracy
train_prediction = model.predict(content_train)
train_data_accuracy = accuracy_score(train_prediction, content_train_labels)
print("Accuracy score of training data: ", train_data_accuracy)

Accuracy score of training data:  0.971875


In [24]:
#Evaluation data accuracy
eval_prediction = model.predict(content_eval)
eval_data_accuracy = accuracy_score(eval_prediction, content_eval_labels)
print("Accuracy score of evaluation data: ", eval_data_accuracy)

Accuracy score of evaluation data:  0.9550480769230769


In [25]:
news_test_dataset = pd.read_csv('test.csv')

In [26]:
news_test_dataset.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [27]:
news_test_dataset.shape

(5200, 4)

In [28]:
news_test_dataset.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [29]:
news_test_dataset = news_test_dataset.fillna('')

In [30]:
news_test_dataset['content'] = news_test_dataset['author'] + news_test_dataset['title']

In [31]:
news_test_dataset['content'] = news_test_dataset['content'].apply(stemming)

In [32]:
news_test_dataset['content']

0       david streitfeldspect trump loosen tongu purs ...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victortim tebow attempt anoth comeback ...
4           truth broadcast networkkeis report meme war e
                              ...                        
5195    jodi rosenth bangladeshi traffic jam never end...
5196    sheryl gay stolbergjohn kasich sign one abort ...
5197    mike mcphatecalifornia today exactli sushi new...
5198                us marin deploy russian border norway
5199        teddi wayneawkward sex onscreen new york time
Name: content, Length: 5200, dtype: object

In [33]:
test_content = news_test_dataset['content'].values

In [34]:
test_content = vectorizer.transform(test_content)

In [35]:
# print(test_content)

In [36]:
#Prediction on test data
new_data = test_content[10]
prediction = model.predict(new_data)
print(prediction)
if(prediction[0]==0):
    print("News is real")
else:
    print("News is fake")

[1]
News is fake
